# Main Code used for experiments

## The code by this point is very unoptimized and messy, but it's also subject of constant change, so I won't clean it up just yet. Still, you can run it to get the experiment results the same I do.

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import shap
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
from DataStreamGenerator_modified import DataStreamGenerator

In [ ]:
class ModelAnalyzer:
  model = None
  shap_explainers = []
  shap_values = []
  shap_absolute_means = []
  shap_representative_values = []
  model_metrics = []
  data_chunks = []
  shap_means = []
  path = ''

  def __init__(self, model,path=''):
    self.model = model
    self.shap_explainers = []
    self.shap_values = []
    self.shap_absolute_means = []
    self.model_metrics = []
    self.data_chunks = []
    self.path=path



  def train(self,X,Y):
    res = []
    for x,y in zip(X,Y):
      merged = np.concatenate((x, y))
      res.append(merged)
    self.data_chunks.append(res)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3,
                                                	stratify =Y,
                                                	random_state = 13)

    self.model.fit(X_train,y_train)
    self.summary(X_test,y_test)
    self.get_shaps(X_test)

  def summary(self,x,y):
    pred = self.model.predict(x)
    tmp = [
        accuracy_score(y,pred),
        precision_score(y,pred)
    ]
    self.model_metrics.append(tmp)

  def get_shaps(self,x):
    explainer = shap.TreeExplainer(self.model)
    self.shap_explainers.append(explainer(x))
    #Calculate average contribution of each feature across all classes and samples
    #(1000,2,2) - num of samples, num of features, num of classes

    tmp = explainer.shap_values(x)

    tmp = np.abs(tmp)

    tmp = np.mean(tmp,2)
    tmp = np.mean(tmp,0)


    exp = explainer(x)
    res = np.empty((len(exp[0]),len(exp[0][0])),float)
    for i in range(len(exp[0])):
        for j in range(len(exp[0][0])):
          max = np.max(exp.values[:,i,j])
          min = np.min(exp.values[:,i,j])
          avgs = []
          for k in range(len(exp)):
            norm = (exp.values[k,i,j] - min)/(max-min)
            avgs.append(norm*exp.data[k,i])
          avgs = np.array(avgs)
          avgs = avgs.mean()
          res[i][j] = avgs
    self.shap_representative_values.append(res)

    self.shap_absolute_means.append(tmp)

    self.shap_values.append(explainer.shap_values(x))





  def export(self, name):
    shaps = np.array(self.shap_absolute_means)
    shap_class_norms = np.array(self.shap_representative_values)
    metrics = np.array(self.model_metrics)
    res = []
    res2 = []
    for shap, metric in zip(shaps, metrics):
      merged = np.concatenate((metric, shap))
      res.append(merged)
    for r, dif in zip(res,shap_class_norms):
      dif = np.array(dif)
      dif = dif.flatten()
      merged = np.concatenate((r,dif))
      res2.append(merged)
    df = pd.DataFrame(res2)
    df.columns = ['accuracy','precision','X','Y','X_Class1','X_Class2','Y_Class1','Y_Class2']
    plt.title(self.path+name +'_results')
    for col in df.columns:
      plt.plot(range(len(df[col])),df[col],label=col ,marker='o')
    plt.legend()
    plt.savefig(self.path+name+'/all_results.png')
    plt.close()
    plt.title(self.path+name+'_shap_results')
    ax = plt.gca()
    ax.set_ylim([0, 0.5])
    plt.plot(range(len(df['X'])),df['X'],label='X', marker='o')
    plt.plot(range(len(df['Y'])),df['Y'],label='Y', marker='o')
    plt.legend()
    plt.savefig(self.path+name+'/shap_results.png')
    plt.close()

    df.to_csv(self.path+name+'/results.csv')
    with open(self.path+name+'/data.pkl','wb') as file:
      pickle.dump(self.data_chunks,file)





In [ ]:

C = DataStreamGenerator(class_count=2, attribute_count=2, sample_count=10000, noise=False, redunce_variable=False)

In [ ]:
DATASETS = [
    [C.Linear_Abrupt,'linear_abrupt'],
    [C.Linear_Gradual_Rotation,'linear_gradual_rotation'],
    [C.Linear_Sudden_Rotation,'linear_sudden_rotation'],
    [C.Linear_Recurrent_Rotation,'linear_recurrent_rotation']
  ]

MODEL = RandomForestClassifier()
INITIAL_BATCH_SIZES = [1000]
ITER_AMOUNTS = [20] # Each has a thousand samples

In [ ]:
#~30s to train on 10 000 samples
for dataset, typ in DATASETS:
  for init_batch_size in INITIAL_BATCH_SIZES:
    for iters in ITER_AMOUNTS:
      model_analyzer = ModelAnalyzer(MODEL)
      name = typ + str(init_batch_size) +'['+ str(iters)+']'
      C.sample_count = init_batch_size
      X,Y = C.InitBatch(typ,name,False,True)
      model_analyzer.train(X,Y)

      C.sample_count = iters * 1000
      X,Y = dataset(name,False,True)
      for i in range(iters):
        start = (i*1000)
        end = 1000+start
        model_analyzer.train(X[start:end],Y[start:end])
      model_analyzer.export(name)
      f = open("contents",'a')
      f.write(name)
      f.close()

/content/DataStreamGenerator.py:169: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(X, Y, s=area0, c="hotpink", edgecolors="g", label="class0", cmap='viridis', alpha=0.7)
/content/DataStreamGenerator.py:170: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(X, Y, s=area1, c="#88c999", edgecolors="blue", label="class1", cmap='viridis', alpha=0.7)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/content/DataStreamGenerator.py:201: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(X, Y, s=area0, c="hotpink", edgecolors="g", label="class0", cmap='viridis', alpha=0.7)
/content/DataStreamGenerator.py:202: UserWarning

plot_decisionboundary


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
!rm -rf sample_data

In [ ]:
!zip -r results.zip /content